In [1]:
import tensorflow as tf
from flows import NormalRW, DFlow, NVPFlow, LogNormal, GVAR, phase,\
Normal, floatX, MVNormal, MVNormalRW, Linear, LinearChol
import flows

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tensorflow.contrib.distributions import WishartCholesky
import math
from flows.models import VARmodel

np.random.seed(1234)
tf.set_random_seed(1234)

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
!ls CDATA

AUS.csv  FRA.csv  GBR.csv  SYNTH.csv


In [3]:
ccodes = ['SYNTH']#['FRA']#, 'FRA', 'GBR']
datas = ['./CDATA/{}.csv'.format(x) for x in ccodes]

In [4]:
datas[0]

'./CDATA/SYNTH.csv'

In [5]:
datas

['./CDATA/SYNTH.csv']

In [6]:
datas = [pd.read_csv(x, index_col='VARIABLE').values.astype(floatX).T[np.newaxis][:,1:-1] for x in datas]

In [7]:
# datas = [x[:,:,:3] for x in datas]

In [8]:
[data.shape for data in datas]

[(1, 201, 4)]

In [9]:
# scaler = 0.
# for i, data in enumerate(datas):
#     stds = (data[0,1:] - data[0,:-1]).std(axis=0)
#     print(stds)
#     scaler = scaler + stds
#     datas[i] = data
# print('---')
# scaler /= len(datas)
for i in range(len(datas)):
#     datas[i] /= scaler
    data = datas[i]
#     stds = (data[0,1:] - data[0,:-1]).std(axis=0)
#     print(stds)
    data = np.concatenate([data[:,1:], data[:,:-1]], axis=-1)
    datas[i] = data

In [10]:
datas[0][:,:4,:4]

array([[[-0.11572005,  0.33028731, -0.48533585, -0.08578253],
        [-0.01044478, -0.11772974,  0.11016907,  0.15420696],
        [-0.1507666 , -0.11943516, -0.07814954,  0.05169588],
        [-0.09098342, -0.22620126, -0.12264003, -0.03106763]]])

In [11]:
country_data = {c:d for c,d in zip(ccodes, datas)}

In [12]:
datas[0].shape

(1, 200, 8)

In [13]:
d = datas[0][0]
d = pd.DataFrame(d.T, columns=range(d.shape[0]))
datas[0] = d

In [14]:
model = VARmodel(datas[0], name='SYNTH', var_dim=4, current_year=3000.)

ldiag logdens Tensor("SYNTH/rw_priors/PWalk_inf/flows/sub_1:0", shape=(1, 1), dtype=float64)
Tensor("SYNTH/OrdWalk/logdens/OrdWalk/Sum:0", shape=(199,), dtype=float64)
Prior disp: [0.06470318 0.0431459  0.50196272 0.03428343]
Tensor("SYNTH/moments/Squeeze_1:0", shape=(4,), dtype=float64) Tensor("SYNTH/DiagPart:0", shape=(4,), dtype=float64)
Tensor("SYNTH/obs_d_prior_2/Sum:0", shape=(199,), dtype=float64)


In [15]:
prior = tf.reduce_sum(model.priors)

logdensity = tf.reduce_sum(model.logdensities)

In [16]:
kl = logdensity - prior
kl /= 36*160

In [17]:
kls = tf.summary.scalar('KLd', kl)
summary = tf.summary.merge([kls, tf.summary.scalar('prior', prior)] + [model.summaries])

In [18]:
main_op = tf.train.AdamOptimizer(0.0001).minimize(kl)

In [19]:
summary = tf.summary.merge_all()

In [20]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()

In [21]:
init.run()

In [22]:
writer = tf.summary.FileWriter('/home/nikita/tmp/tblogs/synth_varmodel_noscale_highvar')

In [23]:
writer.add_graph(tf.get_default_graph())

In [24]:
epoch = 0

In [43]:
for epoch in range(epoch, 100000):
    for step in range(100):
        sess.run(main_op)
    s, _ = sess.run([summary, main_op])
    writer.add_summary(s, global_step=epoch)

KeyboardInterrupt: 

In [44]:
diags = []
for _ in range(4000):
    d = np.sqrt(np.diag(model.PWalk.sigma.eval()))
    diags.append(d)
diags = np.mean(diags, axis=0)

In [45]:
diags

array([0.01076634, 0.01018942, 0.01164419, 0.00963747, 0.01139492,
       0.01045801, 0.01125562, 0.00979578, 0.00629053, 0.01015415,
       0.01119636, 0.01100517, 0.00970666, 0.01055901, 0.01102877,
       0.01115573, 0.00980632, 0.00686542, 0.00800965, 0.00796904,
       0.01144837, 0.00799516, 0.00807688, 0.00822222, 0.02787968,
       0.00810673, 0.00676385, 0.00963918, 0.00966401, 0.01021532,
       0.00941055, 0.00931863, 0.00961691, 0.01029697, 0.01009124,
       0.00770487])

In [ ]:
wsigmas = tf.sqrt(tf.diag_part(model.PWalk.sigma))

In [ ]:
wsigmas

In [ ]:
global_post=global_inf.output[0]

In [ ]:
ss = []
for _ in range(1000):
    ss.append(wsigmas.eval())
ss = np.array(ss)

In [ ]:
np.mean(ss,axis=0)

In [ ]:
ss.std(axis=0)

In [ ]:
sns.kdeplot(ss[:,4], ss[:,1])
plt.show()